In [ ]:
from openpyxl import Workbook
from bs4 import BeautifulSoup
import urllib.request
import random
import string
import requests

#Utwórz obiekt klasy Workbooka, a następnie utwórz 3 arkusze
wb = Workbook()
ws1 = wb.create_sheet("Giełda")
ws2 = wb.create_sheet(title="Linki")
ws3 = wb.create_sheet("Filmweb")

## pod adresem https://stooq.pl/q/?s=cdr znajdziesz notowania giełdowe spółki CD Projekt Red: wygeneruj 5 losowych kodów 3-literowych ...


In [ ]:
#utworz liste która będzie posiadała pięć elementów
listWithValues = list()

while len(listWithValues) < 5:
    threeLetters  = ''.join(random.choice(string.ascii_lowercase) for i in range(3)) #wygenreuj 3 losowe litery
    page = requests.get("https://stooq.pl/q/?s=" + threeLetters) #utworz reuqest
    link = "https://stooq.pl/q/?s=" + threeLetters #zapisz link do sprawdzenia
    soup = BeautifulSoup(page.text, 'html.parser') #utworz soup z raw textu powyższego requestu

    #znajdz link jeśli istnieje oznacza to że wpsomniane 3 litery nie mają wyniki ale zawierja link do najbliższego wyszukania
    #weź go a następnie zmień sopa na podstawie przybliżonego wyniku
    fontWithLink = soup.find("font", {"id": "f16"}) 
    if not(fontWithLink is None):
        page = requests.get("https://stooq.pl/" + fontWithLink.find('a')['href'])
        link = "https://stooq.pl/" + fontWithLink.find('a')['href']
        soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find("table", {"id": "t1"}) #odnajdź tabele z danymi
    if(table is None):
        continue #jesli jej nie mam omiń tę stronę
    tableWithData = soup.find("table", {"id": "t1"}).find('tbody') #pobierz body tabeli
    children = tableWithData.findChildren("tr" , recursive=False) #znajdź rowy tabeli
    kurs = children[0].find('td').find('span').text 
    zmiana = children[1].find('td').find('span').text
    liczbaTransakcji = children[6].findChildren('td')[1].find('span').text 
    listWithValues.append(kurs + "|" + zmiana + "|" + liczbaTransakcji + '|' + link) #znajdź potrzebne dane i zapisz do listy
  
    #do arkusza pierwszego zapisz elemnty z listy w kolumnie pierwzej od A do F
counter = 1
for dataValue in listWithValues:
    cellref=ws1.cell(row=counter, column=1)
    cellref.value=dataValue
    counter+=1

## dla swojej ulubionej strony internetowej napisz kod, który połączy się ze stroną, znajdzie wszystkie linki na stronie (znacznik ‘a’ posiadający atrybut ‘href’), a następnie zapisze je do arkusza ‘Linki’,

In [ ]:
#jak wyżej pobierz Stronę ale tym razem korzystając z urllib
source = urllib.request.urlopen('https://aros.pl/').read()
soup = BeautifulSoup(source,'lxml')

#znajdź wszystkie a a następnie zapisz do 2 kolumny 2 arkusza wszystkie linki
aTags = soup.find_all('a')
counter = 1
for hhref in aTags:
      cellref=ws2.cell(row=counter, column=2)
      cellref.value=hhref['href']
      counter+=1

## dla ustalonego linku do filmu na Filmwebie, np. https://www.filmweb.pl/film/To%3A+Rozdzia%C5%82+2-2019-793838, napisz kod, który zwróci: reżysera,datę premiery,boxoffice,,ocenę filmu.,zapisz uzyskane wyniki do arkusza ‘Filmweb’

In [ ]:
#pobierz stronę korzystając z urllib i utwórz obiekt soup zawięrający te stronę
source = urllib.request.urlopen('https://www.filmweb.pl/film/To%3A+Rozdzia%C5%82+2-2019-793838').read()
soup = BeautifulSoup(source)
mainTable = soup.find('div', attrs={'class': 'filmInfo'}) #pobierz tabele zawierającą reżyera i boxOffice a następnie go pobierz
children = mainTable.find('table').findChildren("tr" , recursive=False)
rezyser = children[0].find('a').text
boxOffice = children[6].find('td').text

#Pobierz premiere
RokPremiery = soup.find("span", {"class": "halfSize"}).text
#pobierz ocene
ocena = soup.find("span", itemprop="ratingValue").text

#Zapisz w kolumnie 3 w/w dane
cellref=ws3.cell(row=1, column=3)
cellref.value=rezyser

cellref=ws3.cell(row=2, column=3)
cellref.value=boxOffice

cellref=ws3.cell(row=3, column=3)
cellref.value=RokPremiery

cellref=ws3.cell(row=4, column=3)
cellref.value=ocena


In [ ]:
wb.save('Bergmann-165IC.xlsx')